In [76]:
# From dct.py

In [77]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import cv2

In [78]:
# https://stackoverflow.com/questions/9777783/suppress-scientific-notation-in-numpy-when-creating-array-from-nested-list
# prevent numpy exponential 
# notation on print, default False
np.set_printoptions(suppress=True)

In [90]:
from IPython.display import Markdown as md

# From https://stackoverflow.com/questions/5466451/how-can-i-print-literal-curly-brace-characters-in-python-string-and-also-use-fo
# https://stackoverflow.com/a/47310524/4162778
def array_to_latex_matrix(arr):
    converted_str = np.array2string(arr, separator=' &', suppress_small=True).replace('[', '').replace(']', '').replace('\n', '\\\\\n').replace('&\\', '\\')
    
    md_format = "\\begin{{bmatrix}}{0}\\end{{bmatrix}}"
    return md(md_format.format(converted_str))

In [91]:
# From https://en.wikipedia.org/wiki/JPEG#JPEG_codec_example
wiki_raw = np.array([
  [52, 55, 61, 66, 70, 61, 64, 73],
  [63, 59, 55, 90, 109, 85, 69, 72],
  [62, 59, 68, 113, 144, 104, 66, 73],
  [63, 58, 71, 122, 154, 106, 70, 69],
  [67, 61, 68, 104, 126, 88, 68, 70],
  [79, 65, 60, 70, 77, 68, 58, 75],
  [85, 71, 64, 59, 55, 61, 65, 83],
  [87, 79, 69, 68, 65, 76, 78, 94]
])

jpg_low_color = np.array([
  [255, 255, 255, 255,  255, 255, 255, 255],
  [255, 255, 255, 255,  255, 255, 255, 0  ],
  [255, 255, 255, 255,  255, 255, 0  , 0  ],
  [255, 255, 255, 255,  0  , 0  , 0  , 255],
  [255, 255, 255, 0  ,  0  , 255, 255, 255],
  [255, 0  , 0  , 0  ,  255, 255, 255, 255],
  [0  , 255, 255, 255,  255, 255, 255, 255],
  [255, 255, 255, 255,  255, 255, 255, 255],
])

def normalizeImg(img):
  # This normalization is wrong
  # return np.array(img) / 255

  # normalized this way following wiki JPEG
  # and https://stackoverflow.com/questions/31949210/assertion-failed-type-cv-32fc1-type-cv-64fc1-in-dct
  return (np.array(img) - 128) / 128

In [92]:
array_to_latex_matrix(jpg_low_color)

\begin{bmatrix}255 &255 &255 &255 &255 &255 &255 &255 \\
 255 &255 &255 &255 &255 &255 &255 &  0 \\
 255 &255 &255 &255 &255 &255 &  0 &  0 \\
 255 &255 &255 &255 &  0 &  0 &  0 &255 \\
 255 &255 &255 &  0 &  0 &255 &255 &255 \\
 255 &  0 &  0 &  0 &255 &255 &255 &255 \\
   0 &255 &255 &255 &255 &255 &255 &255 \\
 255 &255 &255 &255 &255 &255 &255 &255\end{bmatrix}

In [82]:
wiki_raw_normalized = normalizeImg(wiki_raw)
jpg_low_color_normalized = normalizeImg(jpg_low_color)

In [83]:
q50_table = np.array([
  [16, 11, 10, 16, 24, 40, 51, 61],
  [12, 12, 14, 19, 26, 58, 60, 55],
  [14, 13, 16, 24, 40, 57, 69, 56],
  [14, 17, 22, 29, 51, 87, 80, 62],
  [18, 22, 37, 56, 68, 109, 103, 77],
  [24, 35, 55, 64, 81, 104, 113, 92],
  [49, 64, 78, 87, 103, 121, 120, 101],
  [72, 92, 95, 98, 112, 100, 103, 99]
])

In [84]:
def convert_dct_cv2(img):
  print('Start DCT')
  print(img)
  print('-----------')
  DCT = cv2.dct(img)
  result = np.round(DCT * 128, decimals=2)
  print('Coefficient')
  print(result)
  print('-----------')
  return result

In [85]:
def quantization(coefficient):
  result = np.round(coefficient / q50_table)
  print('After quantization')
  print(result)
  print('-----------')
  return result

In [86]:
wiki_result = quantization(convert_dct_cv2(wiki_raw_normalized))

Start DCT
[[-0.59375   -0.5703125 -0.5234375 -0.484375  -0.453125  -0.5234375
  -0.5       -0.4296875]
 [-0.5078125 -0.5390625 -0.5703125 -0.296875  -0.1484375 -0.3359375
  -0.4609375 -0.4375   ]
 [-0.515625  -0.5390625 -0.46875   -0.1171875  0.125     -0.1875
  -0.484375  -0.4296875]
 [-0.5078125 -0.546875  -0.4453125 -0.046875   0.203125  -0.171875
  -0.453125  -0.4609375]
 [-0.4765625 -0.5234375 -0.46875   -0.1875    -0.015625  -0.3125
  -0.46875   -0.453125 ]
 [-0.3828125 -0.4921875 -0.53125   -0.453125  -0.3984375 -0.46875
  -0.546875  -0.4140625]
 [-0.3359375 -0.4453125 -0.5       -0.5390625 -0.5703125 -0.5234375
  -0.4921875 -0.3515625]
 [-0.3203125 -0.3828125 -0.4609375 -0.46875   -0.4921875 -0.40625
  -0.390625  -0.265625 ]]
-----------
Coefficient
[[-415.38  -30.19  -61.2    27.24   56.12  -20.1    -2.39    0.46]
 [   4.47  -21.86  -60.76   10.25   13.15   -7.09   -8.54    4.88]
 [ -46.83    7.37   77.13  -24.56  -28.91    9.93    5.42   -5.65]
 [ -48.53   12.07   34.1   -14.

In [87]:
jpg_low_color_result = quantization(convert_dct_cv2(jpg_low_color_normalized))

Start DCT
[[ 0.9921875  0.9921875  0.9921875  0.9921875  0.9921875  0.9921875
   0.9921875  0.9921875]
 [ 0.9921875  0.9921875  0.9921875  0.9921875  0.9921875  0.9921875
   0.9921875 -1.       ]
 [ 0.9921875  0.9921875  0.9921875  0.9921875  0.9921875  0.9921875
  -1.        -1.       ]
 [ 0.9921875  0.9921875  0.9921875  0.9921875 -1.        -1.
  -1.         0.9921875]
 [ 0.9921875  0.9921875  0.9921875 -1.        -1.         0.9921875
   0.9921875  0.9921875]
 [ 0.9921875 -1.        -1.        -1.         0.9921875  0.9921875
   0.9921875  0.9921875]
 [-1.         0.9921875  0.9921875  0.9921875  0.9921875  0.9921875
   0.9921875  0.9921875]
 [ 0.9921875  0.9921875  0.9921875  0.9921875  0.9921875  0.9921875
   0.9921875  0.9921875]]
-----------
Coefficient
[[ 633.5    81.69   24.4    28.69  -63.75  -19.17   58.9   -16.25]
 [  16.25  243.87  -90.49   27.83    1.34   46.03    0.85  -25.82]
 [ 259.99  -98.8  -198.98   44.21  -10.11    8.79 -108.83   33.11]
 [ -19.17 -292.67  172.18  

In [97]:
# Remove trailing dot
# https://stackoverflow.com/questions/57980827/is-it-possible-to-remove-the-dot-from-each-of-the-following-values
array_to_latex_matrix(wiki_result.astype(int))

\begin{bmatrix}-26 & -3 & -6 &  2 &  2 & -1 &  0 &  0 \\
   0 & -2 & -4 &  1 &  1 &  0 &  0 &  0 \\
  -3 &  1 &  5 & -1 & -1 &  0 &  0 &  0 \\
  -3 &  1 &  2 & -1 &  0 &  0 &  0 &  0 \\
   1 &  0 &  0 &  0 &  0 &  0 &  0 &  0 \\
   0 &  0 &  0 &  0 &  0 &  0 &  0 &  0 \\
   0 &  0 &  0 &  0 &  0 &  0 &  0 &  0 \\
   0 &  0 &  0 &  0 &  0 &  0 &  0 &  0\end{bmatrix}

In [96]:
array_to_latex_matrix(jpg_low_color_result.astype(int))

\begin{bmatrix} 40 &  7 &  2 &  2 & -3 &  0 &  1 &  0 \\
   1 & 20 & -6 &  1 &  0 &  1 &  0 &  0 \\
  19 & -8 &-12 &  2 &  0 &  0 & -2 &  1 \\
  -1 &-17 &  8 &  4 & -1 &  0 &  0 &  1 \\
   4 &  3 &  6 & -3 &  0 &  0 &  0 & -1 \\
  -1 &  0 & -1 & -3 &  1 & -1 &  0 & -1 \\
  -1 &  0 &  0 &  2 &  1 &  0 &  1 &  0 \\
   1 &  0 & -1 &  0 & -2 & -1 &  0 &  0\end{bmatrix}